In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from math import e
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from keras.models import load_model

In [2]:
# Read Orography
OroData = xr.open_dataset('../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')
# create mask
oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values


In [3]:
#Surface data
t2mData = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_tas_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
rhData  = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_hurs_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
u10Data = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_uas_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
v10Data = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_vas_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')

In [4]:
# Level Data
tLevData = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_ta_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
zLevData = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_zg_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
wLevData = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_wap_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
uLevData = xr.open_dataset('../../Data/CMIP6/INM-CM4-8/processed/Regrid_ua_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')

In [5]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v10Data.vas.values**2)+(u10Data.uas.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhData.hurs/100.0)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

#Calculate inv
inv=t2mData.tas.values-tLevData.ta.sel(plev=85000,method='nearest').values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLevData.ua.sel(plev=85000,method='nearest').values-u10Data.uas.values)/(zLevData.zg.sel(plev=85000,method='nearest').values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [6]:
# AO data
AOData = xr.open_dataset('../March2021/INM-CM4-8/INM-CM4-8-AOindex-NDJF-Daily-1980-2014.nc')
aoTS=AOData.AO
datetimeindex = aoTS.indexes['time'].to_datetimeindex()
datetimeindex
aoTS['time'] = datetimeindex

# EU data
EUData = xr.open_dataset('../March2021/INM-CM4-8/INM-CM4-8-EUindex-NDJF-Daily-1980-2014.nc')
EUData.EU
euTS=EUData.EU
datetimeindex = euTS.indexes['time'].to_datetimeindex()
datetimeindex
euTS['time'] = datetimeindex

# 5Day and 30Day Mean
#AO
AO5D=aoTS.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

#EU
EU5D=euTS.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))


<ipython-input-6-52c5e1759a73>:4: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = aoTS.indexes['time'].to_datetimeindex()
<ipython-input-6-52c5e1759a73>:12: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = euTS.indexes['time'].to_datetimeindex()


In [7]:
t2m=t2mData.tas
ws=ws_ds.ws
rh=rh_ds.rh
inv=inv_ds.inv
w=wLevData.sel(plev=70000,method='nearest').wap
ushear=ushear_ds.ushear
z=zLevData.sel(plev=50000,method='nearest').zg

In [8]:
datetimeindex = t2m.indexes['time'].to_datetimeindex()
t2m['time'] = datetimeindex
ws['time'] = datetimeindex
rh['time'] = datetimeindex
inv['time'] = datetimeindex
w['time'] = datetimeindex
ushear['time'] = datetimeindex
z['time'] = datetimeindex

<ipython-input-8-63868420b6dd>:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = t2m.indexes['time'].to_datetimeindex()


In [9]:
t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))
zTsAll=z[((z.time.dt.month>11) | (z.time.dt.month<2)) & (z.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'),latitude=slice(35,0),longitude=slice(50,100))

In [10]:
t2mTsAll.values=t2mTsAll.values*mask
wsTsAll.values=wsTsAll.values*mask
rhTsAll.values=rhTsAll.values*mask
invTsAll.values=invTsAll.values*mask
ushearTsAll.values=ushearTsAll.values*mask
wTsAll.values=wTsAll.values*mask
zTsAll.values=zTsAll.values*mask

In [11]:
invTsAll=invTsAll.drop_vars('height')
t2mTsAll=t2mTsAll.drop_vars('height')
wsTsAll=wsTsAll.drop_vars('height')
rhTsAll=rhTsAll.drop_vars('height')


In [12]:
wTsAll=wTsAll.drop_vars('plev')
zTsAll=zTsAll.drop_vars('plev')
ds = xr.Dataset(
    data_vars=dict(
        t2m=t2mTsAll.mean(dim='time'),
        ws10m=wsTsAll.mean(dim='time'),
        rh10m=rhTsAll.mean(dim='time'),
        inv=invTsAll.mean(dim='time'),
        ushear=ushearTsAll.mean(dim='time'),
        w700=wTsAll.mean(dim='time'),
        z500=zTsAll.mean(dim='time'),
    )
    
)

In [13]:
ds.to_netcdf('../May2021/INM-CM4-8_metvars.nc')

In [14]:
ds_1 = xr.Dataset(
    data_vars=dict(
        AO=AO5DAll,
        EU=EU5DAll,
    )
)

In [15]:
ds_1.to_netcdf('../May2021/INM-CM4-8_telvars.nc')